In [1]:
import torch

### 행렬 곱

In [3]:
t1 = torch.FloatTensor([[1, 2],
                        [3, 4],
                        [5, 6]])

t2 = torch.FloatTensor([[1, 2], 
                        [1, 2]])


print(t1.size(), t2.size())

torch.Size([3, 2]) torch.Size([2, 2])


In [4]:
z = torch.matmul(t1, t2)
print(z.size())

torch.Size([3, 2])


#### 1. 배치 행렬 곱

In [11]:
t1 = torch.FloatTensor(3, 3, 2)
t2 = torch.FloatTensor(3, 2, 3)

t1dott2= torch.matmul(t1, t2)

print('t1', t1.size())
print('t2', t2.size())
print('t1dott2', t1dott2.size())
print('t1dott3bmm', torch.bmm(t1, t2).size()) # 병렬로 처리

t1 torch.Size([3, 3, 2])
t2 torch.Size([3, 2, 3])
t1dott2 torch.Size([3, 3, 3])
t1dott3bmm torch.Size([3, 3, 3])


In [12]:
t1 = torch.FloatTensor([1, 2, 3])
t1.size()

torch.Size([3])

In [14]:
t2 = torch.FloatTensor([[1], [2], [3]])
t2.size()

torch.Size([3, 1])

In [16]:
print(t1.shape)
print(t2.shape)

torch.Size([3])
torch.Size([3, 1])


### 선형 계층

In [17]:
W = torch.FloatTensor([[1, 2],
                        [3, 4],
                        [5, 6]])
b = torch.FloatTensor([2, 2])

print('W', W.size())
print('b', b.size())

W torch.Size([3, 2])
b torch.Size([2])


In [20]:
# X * W + b

def linear(x, W, b):
    y = torch.matmul(x, W) + b
    return y

In [21]:
x = torch.FloatTensor(4, 3)

y = linear(x, W, b)

print(y)
print(y.size()) # (4x3) (3x2) ==> (4, 2) + (2, ) ==> (4, 2)

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])
torch.Size([4, 2])


#### torch.nn.Module 클래스 상속 받기

In [22]:
from turtle import forward
import torch.nn as nn

class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.W = torch.FloatTensor(input_dim, output_dim)
        self.b = torch.FloatTensor(output_dim)

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b

        # |y| = (batch_size, input_dim) * (input_dim, output_dim)
        #     = (batch_size, output_dim)
        return y

In [23]:
linear = MyLinear(3, 2)

y = linear(x)

print(y.size())
print(y)


torch.Size([4, 2])
tensor([[0.0000e+00, 1.5846e+29],
        [0.0000e+00, 1.5846e+29],
        [0.0000e+00, 1.5846e+29],
        [0.0000e+00, 1.5846e+29]])


#### 올바른 방법: nn.Parameter 활용하기

In [28]:
import torch.nn as nn

class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        # nn.Parameter : 파라미터 정상적으로 출력 (W, b) 학습되는 파라미터로 선언
        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.b = nn.Parameter(torch.FloatTensor(output_dim))

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b

        # |y| = (batch_size, input_dim) * (input_dim, output_dim)
        #     = (batch_size, output_dim)
        return y

In [30]:
linear = MyLinear(3, 2)
y = linear(x)

print(y)
print(y.size())

tensor([[7.0065e-45, 0.0000e+00],
        [7.0065e-45, 0.0000e+00],
        [7.0065e-45, 0.0000e+00],
        [7.0065e-45, 0.0000e+00]], grad_fn=<AddBackward0>)
torch.Size([4, 2])


In [31]:
for i in linear.parameters():
    print(i)

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], requires_grad=True)
Parameter containing:
tensor([7.0065e-45, 0.0000e+00], requires_grad=True)


#### nn.Linear 활용하기

In [34]:
## 미리 정의된 Linear 계층

linear = nn.Linear(3, 2)
y = linear(x)

print(y)
print(y.size(), end='\n\n')

for p in linear.parameters():
    print(p)

tensor([[0.2750, 0.1127],
        [0.2750, 0.1127],
        [0.2750, 0.1127],
        [0.2750, 0.1127]], grad_fn=<AddmmBackward>)
torch.Size([4, 2])

Parameter containing:
tensor([[ 0.3071, -0.1727, -0.5293],
        [-0.3094, -0.1949,  0.1925]], requires_grad=True)
Parameter containing:
tensor([0.2750, 0.1127], requires_grad=True)


In [36]:
import torch.nn as nn

class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        # 미리 정의된 Linear 선형 계층 사용
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = self.linear(x)

        # |y| = (batch_size, input_dim) * (input_dim, output_dim)
        #     = (batch_size, output_dim)
        return y

In [38]:
linear = MyLinear(3, 2)

y = linear(x)
print(y)
print(y.size())

tensor([[-0.4222, -0.4757],
        [-0.4222, -0.4757],
        [-0.4222, -0.4757],
        [-0.4222, -0.4757]], grad_fn=<AddmmBackward>)
torch.Size([4, 2])


### GPU 사용하기

    `157p`

In [39]:
# x = torch.cuda.FloatTensor(2, 2)
# x

TypeError: type torch.cuda.FloatTensor not available. Torch not compiled with CUDA enabled.